Импортируем библиотеки

In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import VBox
from scipy.interpolate import interp1d, lagrange, CubicSpline, interp1d, pchip
from scipy.interpolate import BarycentricInterpolator, Akima1DInterpolator
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tsa.holtwinters import ExponentialSmoothing
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

Создаём Pandas DataFrame из файла IBM.csv

In [9]:
file_path = 'MSFT.csv'
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df["Volume"] = df["Volume"] / max(df["Volume"])

Отрисуем график стоимости акций

In [10]:
trace1 = go.Scatter(x=df.index, y=df['Volume'], mode="lines+markers", name='Исходные данные')
fig1 = go.FigureWidget(layout_yaxis_range=[0,np.max(df['Volume'])*1.1])
fig1.add_trace(trace1)
fig1.update_xaxes(title='Дата')
fig1.update_yaxes(title='Стоимость')
fig1

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': '132b67f3-4ca8-4e94-9831-7b1013b2b0a7',
              'x': array([datetime.datetime(2022, 9, 6, 0, 0),
                          datetime.datetime(2022, 9, 7, 0, 0),
                          datetime.datetime(2022, 9, 8, 0, 0), ...,
                          datetime.datetime(2023, 8, 31, 0, 0),
                          datetime.datetime(2023, 9, 1, 0, 0),
                          datetime.datetime(2023, 9, 5, 0, 0)], dtype=object),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'Дата'}},
               'yaxis': {'range': [0, 1.1], 'title': {'text': 'Стоимость'}}}
})

### ARIMA

In [11]:

# Прогнозирование с использованием ARIMA
order = (5, 1, 1)  # параметры модели ARIMA (p, d, q)
model = ARIMA(df['Volume'], order=order)
result = model.fit()

forecast_steps = 10 
forecast = result.get_forecast(steps=forecast_steps)

forecast_values = forecast.predicted_mean
# Визуализация результатов
trace2 = go.Scatter(
    x=pd.date_range(start=df.index[-1], periods=forecast_steps + 1, freq='B')[1:],
    y=forecast_values,
    mode='markers',
    name='ARIMA'
)

fig2 = go.FigureWidget(data=[trace1, trace2])
fig2.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_arima(**args):
    p, d, q, end, forecast_n = args.values()
    
    order = (p, d, q)  # параметры модели ARIMA (p, d, q)
    model = ARIMA(df['Volume'][:end], order=order)
    result = model.fit()

    forecast = result.get_forecast(steps=forecast_n)
    
    forecast_values = forecast.predicted_mean
    
    mse_arima = mean_squared_error(df['Volume'][end:end+forecast_n], forecast_values)
    mae_arima = mean_absolute_error(df['Volume'][end:end+forecast_n], forecast_values)
    
    fig2["data"][1].x = df.index[end:end+forecast_n]
    fig2["data"][1].y = forecast_values
    
    print(f'MSE (ARIMA): {mse_arima}')
    print(f'MAE (ARIMA): {mae_arima}')


interact(
    redraw_arima, 
    int1=widgets.IntSlider(value=5, min=0, max=10, step=1,description='p:'), 
    int2=widgets.IntSlider(value=1, min=0, max=10, step=1,description='d:'),
    int3=widgets.IntSlider(value=1, min=0, max=10, step=1,description='q:'), 
    int4=widgets.IntSlider(value=1, min=1, max=len(df.index), step=1,description='end:'), 
    int5=widgets.IntSlider(value=1, min=1, max=100, step=1,description='forecast_n:')
)

# Вывод результатов
fig2

interactive(children=(IntSlider(value=5, description='p:', max=10), IntSlider(value=1, description='d:', max=1…

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': 'cf123eab-6d42-495b-ad0e-18a6df4449c3',
              'x': array([datetime.datetime(2022, 9, 6, 0, 0),
                          datetime.datetime(2022, 9, 7, 0, 0),
                          datetime.datetime(2022, 9, 8, 0, 0), ...,
                          datetime.datetime(2023, 8, 31, 0, 0),
                          datetime.datetime(2023, 9, 1, 0, 0),
                          datetime.datetime(2023, 9, 5, 0, 0)], dtype=object),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])},
             {'mode': 'markers',
              'name': 'ARIMA',
              'type': 'scatter',
              'uid': '4225d0e3-bbd5-446d-9957-a13b64984f07',
              'x': array([datetime.datetime(2022, 9, 7, 0, 0)], dtype=object),
              'y': array([0.24770853])}],


### VAR

In [ ]:
order = 2  # порядок VAR-модели
model_var = VAR(df)
result_var = model_var.fit(order)

forecast_steps = 10 
lag_order = result_var.k_ar
forecast = result_var.forecast(df.values[-lag_order:], steps=forecast_steps)

# Визуализация результатов
trace2 = go.Scatter(
    x=pd.date_range(start=df.index[-1], periods=forecast_steps + 1, freq='B')[1:],
    y=forecast[:, -1],
    mode='markers',
    name='VAR'
)

fig3 = go.FigureWidget(data=[trace1, trace2])
fig3.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_var(**args):
    order, end, forecast_n = args.values()

    model_var = VAR(df)
    result_var = model_var.fit(order)
    forecast = result_var.forecast(model_var.y[:end], steps=forecast_n)
    forecast_values = forecast[:,-1]
    
    mse_arima = mean_squared_error(df['Volume'][end:end+forecast_n], forecast_values)
    mae_arima = mean_absolute_error(df['Volume'][end:end+forecast_n], forecast_values)
    
    fig3["data"][1].x = df.index[end:end+forecast_n]
    fig3["data"][1].y = forecast_values
    
    print(f'MSE (VAR): {mse_arima}')
    print(f'MAE (VAR): {mae_arima}')


interact(
    redraw_var, 
    int1=widgets.IntSlider(value=5, min=0, max=10, step=1,description='p:'), 
    int2=widgets.IntSlider(value=1, min=1, max=len(df.index), step=1,description='end:'), 
    int3=widgets.IntSlider(value=1, min=1, max=100, step=1,description='forecast_n:')
)

# Вывод результатов
fig3

### Holt Winter’s Exponential Smoothing

In [14]:
hw_model = ExponentialSmoothing(df['Volume'], trend='add', seasonal='add', seasonal_periods=12)
hw_result = hw_model.fit()

forecast_steps = 10 
forecast = hw_result.forecast(steps=forecast_steps)

# Визуализация результатов
trace2 = go.Scatter(
    x=pd.date_range(start=df.index[-1], periods=forecast_steps + 1, freq='B')[1:],
    y=forecast,
    mode='markers',
    name='HWES'
)

fig4 = go.FigureWidget(data=[trace1, trace2])
fig4.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def redraw_var(**args):
    order, end, forecast_n = args.values()

    hw_model = ExponentialSmoothing(df['Volume'][:end], trend='add', seasonal='add', seasonal_periods=order)
    hw_result = hw_model.fit()

    forecast = hw_result.forecast(steps=forecast_n)
    
    mse_arima = mean_squared_error(df['Volume'][end:end+forecast_n], forecast)
    mae_arima = mean_absolute_error(df['Volume'][end:end+forecast_n], forecast)
    
    fig4["data"][1].x = df.index[end:end+forecast_n]
    fig4["data"][1].y = forecast
    
    print(f'MSE (HWES): {mse_arima}')
    print(f'MAE (HWES): {mae_arima}')


interact(
    redraw_var, 
    int1=widgets.IntSlider(value=5, min=1, max=100, step=1,description='p:'), 
    int2=widgets.IntSlider(value=2, min=2, max=len(df.index), step=1,description='end:'), 
    int3=widgets.IntSlider(value=1, min=1, max=100, step=1,description='forecast_n:')
)

# Вывод результатов
fig4

interactive(children=(IntSlider(value=5, description='p:', min=1), IntSlider(value=2, description='end:', max=…

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': 'a322df85-869a-42d4-bf7b-b1165ece4203',
              'x': array([datetime.datetime(2022, 9, 6, 0, 0),
                          datetime.datetime(2022, 9, 7, 0, 0),
                          datetime.datetime(2022, 9, 8, 0, 0), ...,
                          datetime.datetime(2023, 8, 31, 0, 0),
                          datetime.datetime(2023, 9, 1, 0, 0),
                          datetime.datetime(2023, 9, 5, 0, 0)], dtype=object),
              'y': array([0.24770853, 0.28021068, 0.235998  , ..., 0.30674084, 0.17341293,
                          0.21531904])},
             {'mode': 'markers',
              'name': 'HWES',
              'type': 'scatter',
              'uid': 'e0ec1f4c-151c-4bd6-bad2-d226ddd6cfe1',
              'x': array([datetime.datetime(2023, 9, 6, 0, 0),
                          datetime.datetime(2023, 9, 7, 0, 